# Import thư viện

In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00


In [2]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1bf1d983362f9611d6fbb3dcab5820cc4bb0c22726878e381ab0876e23357069
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [3]:
pip install pyvi spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00


In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import sys
sys.path.append('/content/drive/MyDrive/DS201')


In [8]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import spacy
from pyvi import ViTokenizer
from collections import Counter
from data_utils.build_vocab import Vocabulary,tokenize_en,tokenize_vi
from data_utils.make_pytorch_dataset import TranslationDataset, MyCollate
from models.bahdanau_encode_decode_model import Encoder, DecoderWithAttention, Seq2Seq
from training import train_model, eval_model
from torch import nn
import torch.optim as optim
import os


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
train_ds = pd.read_json(r'/content/drive/MyDrive/DS201/small-PhoMT/small-train.json')
dev_ds = pd.read_json(r'/content/drive/MyDrive/DS201/small-PhoMT/small-dev.json')
test_ds = pd.read_json(r'/content/drive/MyDrive/DS201/small-PhoMT/small-test.json')

In [10]:
train_ds

,english,vietnamese
0,It begins with a countdown .,Câu chuyện bắt đầu với buổi lễ đếm ngược .
1,"On August 14th , 1947 , a woman in Bombay goes...","Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở..."
2,"Across India , people hold their breath for th...","Cùng lúc , trên khắp đất Ấn , người ta nín thở..."
3,"And at the stroke of midnight , a squirming in...","Khi đồng hồ điểm thời khắc nửa đêm , một đứa t..."
4,"These events form the foundation of "" Midnight...","Những sự kiện này là nền móng tạo nên "" Những ..."
...,...,...
19995,And the man was incredibly curious .,Và người đàn ông này cực kỳ tò mò .
19996,And he wanted to understand what it was and wh...,Và ông muốn hiểu nó là gì và tại sao thế rằng ...
19997,"And one day , we were walking .","Một ngày nọ , chúng tôi đang đi bộ ."
19998,"We were in France , in Les Houches .","Chúng tôi ở Pháp , tại Les Houches ."


In [11]:
dev_ds

,english,vietnamese
0,"﻿Hurricane Dorian , one of the most powerful s...","Vào chủ nhật ngày 1-9-2019 , cơn bão Dorian , ..."
1,Dorian is especially dangerous due to its slow...,Bão Dorian đặc biệt nguy hiểm vì nó di chuyển ...
2,"The storm passed by the Leeward Islands , Puer...","Khi đi qua quần đảo Leeward , Puerto Rico và q..."
3,The United States branch office continues to g...,Văn phòng chi nhánh Hoa Kỳ tiếp tục cập nhật t...
4,"At this time , there have been no reported inj...","Theo báo cáo đến thời điểm hiện tại , trong 46..."
...,...,...
1995,You could even imagine a version of this scena...,Một dị bản của viễn cảnh này là nơi mà mọi ngư...
1996,"Over the course of last year , open - source h...","Năm ngoái , các hacker chuyên về ổ cứng mã ngu..."
1997,"At the other end of the network , there 'd be ...","Ở đầu kia của mạng lưới , sẽ có dịch vụ giúp c..."
1998,"Now , you Web 2.0 folks in the audience know w...",Những ai ở đây thuộc thế hệ Web 2.0 sẽ hiểu tô...


In [12]:
test_ds

,english,vietnamese
0,"Brother Albert Barnett and his wife , Sister S...","Anh Albert Barnett và chị Susan Barnett , thuộ..."
1,Severe storms ripped through parts of the sout...,"Ngày 11 và 12-1-2020 , những cơn bão lớn đã qu..."
2,"Two days of heavy rain , high winds , and nume...",Những trận mưa to và gió lớn trong suốt hai ng...
3,"Sadly , Brother Albert Barnett and his wife , ...","Đáng buồn là anh Albert Barnett 85 tuổi , và v..."
4,The United States branch also reports that at ...,Chi nhánh Hoa Kỳ cũng cho biết có ít nhất bốn ...
...,...,...
1995,Toyota applied the principles of modularity of...,Toyota áp dụng các nguyên tắc của tính đơn lẻ ...
1996,"Now fortunately , few companies succumb to cat...",Thật may là một vài công ti không chống cự ngọ...
1997,But we do read in the newspaper every day abou...,Nhưng chúng ta đọc báo chí mỗi ngày về các côn...
1998,"How is it , then , that the consumer optics gi...",Sau đó thì gã khổng lồ tiêu dùng quang học có ...


# Xây dựng vocab và pytorch dataset

In [13]:
BATCH_SIZE = 32


In [14]:
# Đường dẫn file muốn lưu
file_vocab_en = "/content/drive/MyDrive/DS201/vocab_en_new.pth"
file_vocab_vi = "/content/drive/MyDrive/DS201/vocab_vi_new.pth"

# Khởi tạo object rỗng
vocab_en = Vocabulary(freq_threshold=4)
vocab_vi = Vocabulary(freq_threshold=4)

# --- XỬ LÝ CHO TIẾNG ANH ---
if os.path.exists(file_vocab_en):
    # TRƯỜNG HỢP 1: Đã có file -> Load lên
    print("Đang tải từ điển tiếng Anh từ file...")
    vocab_en.load_vocab(file_vocab_en)
else:
    # TRƯỜNG HỢP 2: Chưa có file -> Build rồi Save
    print("Đang xây dựng từ điển tiếng Anh (lần đầu)...")
    vocab_en.build_vocabulary(train_ds['english'].tolist(), tokenize_en)
    vocab_en.save_vocab(file_vocab_en)

# --- XỬ LÝ CHO TIẾNG VIỆT ---
if os.path.exists(file_vocab_vi):
    print("Đang tải từ điển tiếng Việt từ file...")
    vocab_vi.load_vocab(file_vocab_vi)
else:
    print("Đang xây dựng từ điển tiếng Việt (lần đầu)...")
    vocab_vi.build_vocabulary(train_ds['vietnamese'].tolist(), tokenize_vi)
    vocab_vi.save_vocab(file_vocab_vi)

print("Hoàn tất chuẩn bị dữ liệu!")

print(f"Kích thước từ điển Anh: {len(vocab_en)}")
print(f"Kích thước từ điển Việt: {len(vocab_vi)}")

dataset_train = TranslationDataset(train_ds, vocab_en, vocab_vi)
dataset_dev = TranslationDataset(dev_ds, vocab_en, vocab_vi)
dataset_test = TranslationDataset(test_ds, vocab_en, vocab_vi)
# Padding index là 0 (đã định nghĩa trong Vocabulary class)
pad_idx = vocab_en.stoi["<pad>"]

# Tạo DataLoader
train_loader = DataLoader(
    dataset=dataset_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=MyCollate(pad_idx=pad_idx)
)

dev_loader = DataLoader(
    dataset=dataset_dev,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=MyCollate(pad_idx=pad_idx)
)

test_loader = DataLoader(
    dataset=dataset_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=MyCollate(pad_idx=pad_idx)
)





Đang tải từ điển tiếng Anh từ file...
Đã tải từ điển từ: /content/drive/MyDrive/DS201/vocab_en_new.pth (Size: 5985)
Đang tải từ điển tiếng Việt từ file...
Đã tải từ điển từ: /content/drive/MyDrive/DS201/vocab_vi_new.pth (Size: 4913)
Hoàn tất chuẩn bị dữ liệu!
Kích thước từ điển Anh: 5985
Kích thước từ điển Việt: 4913


# Định nghĩa các siêu tham số

In [15]:
MAX_LENGTH = 50
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_vi)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HIDDEN_DIM = 256   # Yêu cầu
N_LAYERS = 3       # Yêu cầu
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_EPOCHS = 10
CLIP = 1

# Khởi tạo mô hình

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT)
dec = DecoderWithAttention(OUTPUT_DIM, DEC_EMB_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)

vocab_trg_stoi = vocab_vi.stoi
vocab_trg_itos = vocab_vi.itos

# Train and Evaluate

In [17]:

print("Bắt đầu huấn luyện...")
for epoch in range(N_EPOCHS):
    train_loss = train_model(model, train_loader, optimizer, criterion, CLIP, device)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')
    rouge_l_score = eval_model(model, dev_loader, vocab_trg_stoi, vocab_trg_itos,device)
    print(f'Epoch: {epoch+1:02} | ROUGE-L Score: {rouge_l_score['rougeL']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge1']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge2']:.2f}')


Bắt đầu huấn luyện...
Epoch: 01 | Train Loss: 6.011
Epoch: 01 | ROUGE-L Score: 0.07 | ROUGE-1-Score: 0.08 | ROUGE-1-Score: 0.00
Epoch: 02 | Train Loss: 5.803
Epoch: 02 | ROUGE-L Score: 0.16 | ROUGE-1-Score: 0.18 | ROUGE-1-Score: 0.04
Epoch: 03 | Train Loss: 5.673
Epoch: 03 | ROUGE-L Score: 0.18 | ROUGE-1-Score: 0.21 | ROUGE-1-Score: 0.05
Epoch: 04 | Train Loss: 5.553
Epoch: 04 | ROUGE-L Score: 0.20 | ROUGE-1-Score: 0.24 | ROUGE-1-Score: 0.06
Epoch: 05 | Train Loss: 5.461
Epoch: 05 | ROUGE-L Score: 0.20 | ROUGE-1-Score: 0.23 | ROUGE-1-Score: 0.06
Epoch: 06 | Train Loss: 5.382
Epoch: 06 | ROUGE-L Score: 0.23 | ROUGE-1-Score: 0.28 | ROUGE-1-Score: 0.08
Epoch: 07 | Train Loss: 5.302
Epoch: 07 | ROUGE-L Score: 0.23 | ROUGE-1-Score: 0.28 | ROUGE-1-Score: 0.08
Epoch: 08 | Train Loss: 5.238
Epoch: 08 | ROUGE-L Score: 0.23 | ROUGE-1-Score: 0.27 | ROUGE-1-Score: 0.08
Epoch: 09 | Train Loss: 5.173
Epoch: 09 | ROUGE-L Score: 0.23 | ROUGE-1-Score: 0.28 | ROUGE-1-Score: 0.09
Epoch: 10 | Train Loss: 

# Test

In [18]:
rouge_l_score = eval_model(model, test_loader, vocab_trg_stoi, vocab_trg_itos,device)
print(f'Test : ROUGE-L Score: {rouge_l_score['rougeL']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge1']:.2f} | ROUGE-1-Score: {rouge_l_score['rouge2']:.2f}')

Test : ROUGE-L Score: 0.25 | ROUGE-1-Score: 0.30 | ROUGE-1-Score: 0.09
